In [3]:
#|default_exp datasets

In [31]:
#|export
from __future__ import annotations
import math,numpy as np,matplotlib.pyplot as plt
from operator import itemgetter
from itertools import zip_longest
import fastcore.all as fc
from torch.utils.data import default_collate,DataLoader
from miniai.training import *
import torchvision.transforms.functional as TF

In [19]:
from datasets import load_dataset
import torch

### Hugging Face dataset

In [11]:
name = 'fashion_mnist'

In [12]:
#|export
def inplace(f):
    def _f(b):
        f(b)
        return b
    return _f

In [15]:
rds = load_dataset(name)

Found cached dataset fashion_mnist (/Users/arun/.cache/huggingface/datasets/fashion_mnist/fashion_mnist/1.0.0/0a671f063342996f19779d38c0ab4abef9c64f757b35af8134b331c294d7ba48)


  0%|          | 0/2 [00:00<?, ?it/s]

In [23]:
x,y = rds['train'].features

In [25]:
@inplace
def transformi(b):
    b[x] = [torch.flatten(TF.to_tensor(ele)) for ele in b[x]]

In [26]:
tds = rds.with_transform(transformi)

In [28]:
#|export
def collated_dict(ds):
    get = itemgetter(*ds.features)
    def _f(b):
        return get(default_collate(b))
    return _f

In [34]:
dlf = DataLoader(tds['train'], batch_size=4, collate_fn=collated_dict(tds['train']))

### Plotting images

In [37]:
#|export
@fc.delegates(plt.Axes.imshow)
def show_image(im, ax=None, figsize=None, title=None, noframe=True, **kwargs):
    if fc.hasattrs(im, ('cpu', 'permute', 'detach')):
        im = im.detach().cpu()
        if len(im.shape)==3 and im.shape[0]<5:
            im = im.permute(1,2,0)
    elif not isinstance(im,np.ndarray):
        im = np.array(im)
    if im.shape[-1]==1:
        im = im[...,0]
    if ax is None:
        _,ax = plt.subplots(figsize=figsize)
    ax.imshow(im, **kwargs)
    if title is not None:
        ax.set_title(title)
    ax.set_xticks([])
    ax.set_yticks([])
    if noframe:
        ax.axis('off')
    return ax